### FE621 - Homework #2

**Author**: Sid Bhatia

**Date**: March 4th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Sveinn Olafsson

**TA**: Dong Woo Kim

#### Problem #1

a. Provide the formulas for the parameters $p, u, d$ of a **Cox-Ross-Rubinstein (CRR) binomial tree** that approximates the dynamics of a stock following *geometric Brownian motion* (*GBM*) with drift $r$ and volatility $\sigma$.